In [1]:
import json, folium, re
import pandas as pd
import numpy as np

In [2]:
park = pd.read_csv('data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17525 entries, 0 to 17524
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          17525 non-null  object 
 1   공원명           17525 non-null  object 
 2   공원구분          17525 non-null  object 
 3   소재지도로명주소      3692 non-null   object 
 4   소재지지번주소       17335 non-null  object 
 5   위도            17513 non-null  float64
 6   경도            17514 non-null  float64
 7   공원면적          17525 non-null  float64
 8   공원보유시설(운동시설)  4821 non-null   object 
 9   공원보유시설(유희시설)  6200 non-null   object 
 10  공원보유시설(편익시설)  4842 non-null   object 
 11  공원보유시설(교양시설)  805 non-null    object 
 12  공원보유시설(기타시설)  2577 non-null   object 
 13  지정고시일         14451 non-null  object 
 14  관리기관명         16033 non-null  object 
 15  전화번호          16343 non-null  object 
 16  데이터기준일자       17525 non-null  object 
 17  제공기관코드        17525 non-null  object 
 18  제공기관명         17525 non-nu

In [3]:
park = park[['공원명','소재지지번주소','공원면적']]
park.dropna(how='any', inplace=True)
park.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [4]:
park = park[park.소재지지번주소.str.contains('경기도')]
park.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


In [5]:
pop = pd.read_csv('data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]

In [6]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [7]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [8]:
park['시군구'] = park.소재지지번주소.apply(lambda x: 
                    x.split()[1] if x.split()[2][-1] != '구' else ' '.join(x.split()[1:3]))
park.tail()

,공원명,소재지지번주소,공원면적,시군구
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시 기흥구
17258,청석공원,경기도 광주시 경안동 242,193530.0,광주시
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,광주시
17261,중대물빛공원,경기도 광주시 중대동 246-2,230932.0,광주시


In [9]:
print(park.시군구.isna().sum())
park.tail()

0


,공원명,소재지지번주소,공원면적,시군구
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시 기흥구
17258,청석공원,경기도 광주시 경안동 242,193530.0,광주시
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,광주시
17261,중대물빛공원,경기도 광주시 중대동 246-2,230932.0,광주시


In [10]:
df = park.pivot_table('공원면적', '시군구', aggfunc='sum')
df.head()

,공원면적
시군구,
가평군,511674.5
고양시 덕양구,3287291.4
고양시 일산동구,2722980.0
고양시 일산서구,922486.9
과천시,315666.0


In [11]:
df = df.merge(pop, left_index=True, right_on='시군구')
df['인당 공원면적'] = (df.공원면적 / df['총 인구수']).round(1)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,시군구,인당 공원면적
1,511674.5,시군,경기도 가평군,62197,가평군,8.2
9,3287291.4,구,경기도 고양시 덕양구,487874,고양시 덕양구,6.7
31,2722980.0,구,경기도 고양시 일산동구,296590,고양시 일산동구,9.2
44,922486.9,구,경기도 고양시 일산서구,290738,고양시 일산서구,3.2
56,315666.0,시군,경기도 과천시,77775,과천시,4.1


In [12]:
df.시군구 = df.시군구.str.replace(' ','')
df.set_index('시군구', inplace=True)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,인당 공원면적
시군구,,,,,
가평군,511674.5,시군,경기도 가평군,62197,8.2
고양시덕양구,3287291.4,구,경기도 고양시 덕양구,487874,6.7
고양시일산동구,2722980.0,구,경기도 고양시 일산동구,296590,9.2
고양시일산서구,922486.9,구,경기도 고양시 일산서구,290738,3.2
과천시,315666.0,시군,경기도 과천시,77775,4.1
